In [1]:
from __future__ import division
import nltk
import re
import matplotlib.pyplot as plt
import numpy as np
from nltk.corpus import brown
from nltk import bigrams, ngrams, trigrams 
from nltk.probability import FreqDist as FreqDist  
from numpy.polynomial.polynomial import polyfit
from collections import Counter
from string import digits
from scipy import stats
import operator
from collections import OrderedDict

In [2]:
from nltk.corpus import treebank,brown

corpus = brown.tagged_sents(tagset='universal')[:-100] 

tag_dict={}
word_dict={}

for sent in corpus:
    for elem in sent:
        w = elem[0]
        tag= elem[1]

        if w not in word_dict:
            word_dict[w]=0

        if tag not in tag_dict:
            tag_dict[tag]=0

        word_dict[w]+=1
        tag_dict[tag]+=1

print(len(word_dict))
print(len(tag_dict))
        
test_data= brown.tagged_sents(tagset='universal')[-10:]

print(len(test_data))

55907
12
10


In [6]:
corpus = brown.tagged_sents(tagset='universal')[:-100] 
corpus[0]

[('The', 'DET'),
 ('Fulton', 'NOUN'),
 ('County', 'NOUN'),
 ('Grand', 'ADJ'),
 ('Jury', 'NOUN'),
 ('said', 'VERB'),
 ('Friday', 'NOUN'),
 ('an', 'DET'),
 ('investigation', 'NOUN'),
 ('of', 'ADP'),
 ("Atlanta's", 'NOUN'),
 ('recent', 'ADJ'),
 ('primary', 'NOUN'),
 ('election', 'NOUN'),
 ('produced', 'VERB'),
 ('``', '.'),
 ('no', 'DET'),
 ('evidence', 'NOUN'),
 ("''", '.'),
 ('that', 'ADP'),
 ('any', 'DET'),
 ('irregularities', 'NOUN'),
 ('took', 'VERB'),
 ('place', 'NOUN'),
 ('.', '.')]

In [31]:
start={}
transition={}
emission={}
context={}
for line in corpus:
    previous='<s>'
    for word,tag in line:
        if(previous=='<s>'):                           # First tag of the sentence
            if tag not in context:
                context[tag]=0
            context[tag]+=1
            if tag not in start:
                start[tag]=0.0
            start[tag]+=1
            previous=tag
        else:                                          # Rest of the word-tag pairs
            if tag not in context:
                context[tag]=0
            context[tag]+=1
            if previous not in transition:
                transition[previous]={}
            if tag not in transition[previous]:
                transition[previous][tag]=0
            transition[previous][tag]+=1
            previous=tag
        if tag not in emission:
            emission[tag]={}
        if word not in emission[tag]:
            emission[tag][word]=0
        emission[tag][word]+=1
    if '</s>' not in transition[previous]:
        transition[previous]['</s>']=0
    transition[previous]['</s>']+=1

for tag in transition:                                 # Normalizing the counts into probability scores
    total_count=float(sum(transition[tag].values())) 
    for next_tag in transition[tag]:
        transition[tag][next_tag]/=(total_count)
for tag in start:                                      # Normalizing the counts into probability scores
    start[tag]/=float(len(corpus))

for tag in emission:                                   # Normalizing the counts into probability scores
    total_count=float(sum(emission[tag].values())) 
    for word in emission[tag]:
        emission[tag][word]/=(total_count)        

In [33]:
transition

{'DET': {'NOUN': 0.6263860039203066,
  'VERB': 0.06470700096544864,
  'DET': 0.005917029928907872,
  'ADJ': 0.23980427723004008,
  'PRON': 0.009895848570843451,
  'NUM': 0.009764196483426465,
  'ADV': 0.017502413621602646,
  'ADP': 0.009091308036628536,
  '.': 0.012748310464878149,
  'X': 0.0013969749275913519,
  'PRT': 0.002011351335537287,
  '</s>': 0.00013165208741698604,
  'CONJ': 0.0006436324273719318},
 'NOUN': {'NOUN': 0.14934835953830775,
  'ADJ': 0.012894665091338726,
  'VERB': 0.1589093883486322,
  'DET': 0.01550122693810779,
  'ADP': 0.24437698809415614,
  '.': 0.2835844769608289,
  'ADV': 0.026370989730073617,
  'CONJ': 0.0596782695628465,
  'PRT': 0.01782786512769245,
  'NUM': 0.008077796964464238,
  'PRON': 0.01978369535581205,
  '</s>': 0.0033154594201581385,
  'X': 0.0003308188675815687},
 'ADJ': {'NOUN': 0.65281583134923,
  '.': 0.10033380792285328,
  'ADP': 0.08844115289360022,
  'CONJ': 0.037604240198131154,
  'PRT': 0.019286679987078403,
  'ADJ': 0.05690288462688889

In [30]:
emission['NOUN']

{'Fulton': 6.18013269108425e-05,
 'County': 0.00030537126238298646,
 'Jury': 1.4541488684904118e-05,
 'Friday': 0.00021812233027356176,
 'investigation': 0.00015632100336271927,
 "Atlanta's": 1.4541488684904118e-05,
 'primary': 5.453058256839044e-05,
 'election': 0.0002617467963282741,
 'evidence': 0.0007270744342452058,
 'irregularities': 2.9082977369808235e-05,
 'place': 0.0017268017813323639,
 'jury': 0.00022902844678723984,
 'term-end': 3.6353721712260294e-06,
 'presentments': 3.6353721712260294e-06,
 'City': 0.0004835044987730619,
 'Committee': 0.00031991275106789055,
 'charge': 0.0003817140779787331,
 'praise': 4.725983822593838e-05,
 'thanks': 9.815504862310279e-05,
 'Atlanta': 0.00012723802599291103,
 'manner': 0.0004507861492320276,
 'September-October': 3.6353721712260294e-06,
 'term': 0.0002726529128419522,
 'Court': 0.0003998909388348632,
 'Judge': 0.00013814414250658911,
 'Durwood': 3.6353721712260294e-06,
 'Pye': 3.6353721712260294e-06,
 'reports': 0.0002362991911296919,
